In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## 配置GPU

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 超参数

In [3]:
sequence_length = 28
input_size = 28
hidden_size = 28
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

## MNIST数据集

In [4]:
train_dataset = torchvision.datasets.FashionMNIST(root='../../data/fashion',
                                                 train=True,
                                                 transform=transforms.ToTensor())
test_dataset = torchvision.datasets.FashionMNIST(root='../../data/fashion',
                                                train=False,
                                                transform=transforms.ToTensor())

## 数据生成器 DataLoader

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [10]:
# RNN网络（多对一）
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)#batch_first：True则输入输出的数据格式为 (batch, seq, feature)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # 设置 隐藏层和神经元初始状态
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # 前向传播LSTM
        out, _ = self.lstm(x, (h0, c0)) # 输出张量形状 (batch_size, seq_length, hidden_size)
        # 解码最后一步的隐藏层状态
        out = self.fc(out[:, -1, :])
        return out

In [11]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

## 损失 和 优化器

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## 训练模型

In [13]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 反向传播 和 优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss: 1.1617
Epoch [1/2], Step [200/600], Loss: 0.7900
Epoch [1/2], Step [300/600], Loss: 0.7127
Epoch [1/2], Step [400/600], Loss: 0.5252
Epoch [1/2], Step [500/600], Loss: 0.6566
Epoch [1/2], Step [600/600], Loss: 0.3681
Epoch [2/2], Step [100/600], Loss: 0.4190
Epoch [2/2], Step [200/600], Loss: 0.3439
Epoch [2/2], Step [300/600], Loss: 0.4833
Epoch [2/2], Step [400/600], Loss: 0.4376
Epoch [2/2], Step [500/600], Loss: 0.4280
Epoch [2/2], Step [600/600], Loss: 0.4791


## 测试模型

In [14]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the 10000 test images: {}%'.format(100 * correct/total))
        

Test Accuracy of the model on the 10000 test images: 84.64%


## 保存模型

In [15]:
torch.save(model.state_dict(), 'model.ckpt')